In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay
import warnings
warnings.filterwarnings("ignore")

import keras
from keras import ops
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.metrics import Mean

import tensorflow as tf
import tensorflow_probability as tfp
import torch

import yfinance as yf
from tqdm import tqdm

In [ ]:
from google.colab import drive
import os

# Mount google drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/Imperial MLDS/DeepTimeSeriesClustering'
os.chdir(path)

In [ ]:
# Run real_data.ipynb
real_data = "real_data.ipynb"
real_data_path = os.path.join(path, real_data)
%run /content/drive/MyDrive/Colab\ Notebooks/Imperial\ MLDS/DeepTimeSeriesClustering/real_data.ipynb

# LSTM-Attention-based Autoencoder

In [ ]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, MultiHeadAttention, LayerNormalization
from keras.regularizers import L2

# Get latent_dim and num_heads from initial config file
latent_dim = config['latent_dim']
num_heads = config['num_heads']

# Encoder
inputs = Input(shape=(time_steps, n_features)) # time_steps and n_features are extracted from real_data.ipynb, but can also be manually assigned
lstm1 = LSTM(latent_dim, return_sequences=True, dropout=0.1, kernel_regularizer=L2(0.03))(inputs)
lstm2 = LSTM(latent_dim, return_sequences=True, dropout=0.1, kernel_regularizer=L2(0.03))(lstm1)
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=latent_dim//num_heads)(lstm2, lstm2) # Multi-Head Self-Attention
attention_output = LayerNormalization()(attention_output + lstm2)
encoder_output = Dense(latent_dim, kernel_regularizer=L2(0.01), activation='relu')(attention_output[:, -1, :]) # Get time_step for latent represenation
encoder = keras.Model(inputs, [encoder_output, attention_output], name='encoder')

encoder.summary()

# Decoder
decoder_inputs = Input(shape=(latent_dim,))
encoder_states = Input(shape=(time_steps, latent_dim)) # latent represenatation
decoded = RepeatVector(time_steps)(decoder_inputs)
# Architecture symmetric to encoder's
decoded = LSTM(latent_dim, return_sequences=True, dropout=0.1, kernel_regularizer=L2(0.03))(decoded)
decoded = LSTM(latent_dim, return_sequences=True, dropout=0.1, kernel_regularizer=L2(0.03))(decoded)
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=latent_dim//num_heads)(decoded, encoder_states)
attention_output = LayerNormalization()(attention_output + decoded)
decoder_output = TimeDistributed(Dense(n_features))(attention_output)
decoder = keras.Model([decoder_inputs, encoder_states], decoder_output, name="decoder")

decoder.summary()


# Deep Momentum Clustering

In [ ]:
from sklearn.cluster import HDBSCAN
from sklearn.metrics import silhouette_score
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import ops, layers

class DeepMomentumClustering(keras.Model):
  def __init__(
      self,
      dataset,
      encoder,
      decoder,
      latent_dim,
      pretrain_epochs=15,
      gamma={
          'mse': 1.0,
          'momentum_guided': 0.3,
          'cluster': 1.0,
          'momentum': 1.0,
          'acceleration': 1.0
      },
      alpha=1.0,
      min_samples=5,
      update_freq=5,
      max_clusters=10,
      lookbacks=[5, 10, 20],
      verbose=1,
      **kwargs
  ):
    super().__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.latent_dim = latent_dim
    self.epsilon = 1e-4
    self.verbose = verbose
    self.min_samples = min_samples
    self.gamma = gamma
    self.alpha = alpha
    self.n_components = max_clusters
    self.max_clusters = max_clusters
    self.lookbacks = lookbacks
    self.update_freq = update_freq
    self.pretrain_epochs = pretrain_epochs
    self.dataset = dataset

    if self.n_components:
      self.cluster_centroids = tf.Variable(
        tf.zeros((self.n_components, self.latent_dim)),
        trainable=True,
        name='cluster_centroids'
      )

    self.current_epoch = tf.Variable(0, trainable=False, dtype=tf.int32)

    # Metrics
    self.loss_metric = keras.metrics.Mean(name='loss')
    self.mse_metric = keras.metrics.Mean(name='mse')
    self.cluster_metric = keras.metrics.Mean(name='cluster')
    self.momentum_metric = keras.metrics.Mean(name='momentum')
    self.acceleration_metric = keras.metrics.Mean(name='acceleration')

    # For momentum and acceleration prediction
    self.momentum_layer = keras.Sequential([
      layers.Dense(16, activation='relu'),
      layers.Dense(1)
    ])
    self.acceleration_layer = keras.Sequential([
      layers.Dense(16, activation='relu'),
      layers.Dense(1)
    ])

  def compute_losses(self, data):
    encoder_outputs = self.encoder(data)
    z = encoder_outputs[0] # Encoder output. Latent representations
    attention_output = encoder_outputs[1]

    x_recon = self.decoder([z, attention_output]) # Reconstruct x using latent representations
    mse = tf.reduce_mean(tf.square(data - x_recon), axis=[1, 2]) # Compute MSE as reconstruction loss

    mask = tf.reduce_any(data != 0, axis=[1, 2]) # Mask for non-padded samples (when not enough time steps)
    mse_loss = tf.where(mask, mse, 0.0)
    mse_loss = tf.reduce_mean(mse_loss)

    z_masked = tf.boolean_mask(z, mask) # Mask for latent representations

    # Clustering loss = 0 if not enough time step or during pretraining
    cluster_loss = tf.cond(
        tf.logical_and(tf.shape(z_masked)[0] > 0, tf.cast(self.current_epoch, tf.float32) >= self.pretrain_epochs),
        lambda: self.clustering_loss(z_masked),
        lambda: tf.constant(0.0, dtype=tf.float32)
    )

    # Clustering weight = 0 during pretraining
    cluster_weight = tf.cond(
      tf.cast(self.current_epoch, tf.float32) < tf.cast(self.pretrain_epochs - 5, tf.float32),
          lambda: tf.constant(0.0, dtype=tf.float32),
          lambda: tf.minimum(
              tf.constant(self.gamma['cluster'], dtype=tf.float32) *
              (tf.cast(self.current_epoch, tf.float32) - tf.cast(self.pretrain_epochs - 5, tf.float32)) / 5.0,
              tf.constant(self.gamma['cluster'], dtype=tf.float32)
          ) # After pretraining, gradually increase weights according to epochs
    )

    mmt_loss = self.momentum_loss(data, z)
    acc_loss = self.acceleration_loss(data, z)

    loss = (self.gamma['mse'] * mse_loss +
                cluster_weight * cluster_loss +
                self.gamma['momentum'] * mmt_loss +
                self.gamma['acceleration'] * acc_loss) # Total loss

    return loss, mse_loss, cluster_loss, mmt_loss, acc_loss

  def clustering_loss(self, z):
    z_exp = tf.expand_dims(z, axis=1)
    centroids_exp = tf.expand_dims(self.cluster_centroids, axis=0)
    centroids_norms = tf.norm(self.cluster_centroids, axis=1)
    valid_centroids = tf.where(centroids_norms > 0)[:, 0]
    centroids_exp = tf.gather(centroids_exp, valid_centroids, axis=1)
    diff = z_exp - centroids_exp
    squared_distances = tf.reduce_sum(tf.square(diff), axis=-1)
    numerator = (1 + squared_distances / self.alpha) ** (-(self.alpha + 1) / 2)
    q = numerator / (tf.reduce_sum(numerator, axis=1, keepdims=True) + self.epsilon)
    f_j = tf.reduce_sum(q, axis=0)
    p_numerator = tf.square(q) / (f_j + self.epsilon)
    p = p_numerator / tf.reduce_sum(p_numerator, axis=1, keepdims=True)
    # KL divergence between target and cluster j's Student's t-distribution
    kl_div = tf.reduce_sum(
        p * (tf.math.log(p + self.epsilon) - tf.math.log(q + self.epsilon)),
        axis=1
    )
    dec_loss = tf.reduce_mean(kl_div)

    momentum_scores = self.compute_momentum_scores(z)
    momentum_guided_loss = self.momentum_guidance_loss(z, q, momentum_scores)
    return dec_loss + self.gamma['momentum_guided'] * momentum_guided_loss

  def compute_momentum_scores(self, z):
    momentum_pred = self.momentum_layer(z)[:, 0]
    momentum_pred = (momentum_pred - tf.reduce_sum(momentum_pred)) / (tf.math.reduce_std(momentum_pred) + self.epsilon)
    return momentum_pred

  def momentum_loss(self, data, z):
    total_momentum_loss = 0.0
    returns = data[:, :, 0]
    volume = data[:, :, 1]

    # Calculate momentum loss for each lookback period
    for lookback in self.lookbacks:
      volume_mean = tf.reduce_mean(volume[:, -lookback:], axis=1, keepdims=True) + self.epsilon
      volume_weights = volume[:, -lookback:] / volume_mean
      volume_weighted_cum_returns = tf.reduce_sum(returns[:, -lookback:] * volume_weights, axis=1)
      volume_weighted_cum_returns = (volume_weighted_cum_returns - tf.reduce_mean(volume_weighted_cum_returns)) / (tf.math.reduce_std(volume_weighted_cum_returns) + self.epsilon) # Normalizing
      momentum_pred = self.momentum_layer(z)[:, 0]
      total_momentum_loss += tf.reduce_mean(tf.square(momentum_pred  - volume_weighted_cum_returns)) / len(self.lookbacks)

    return total_momentum_loss

  def acceleration_loss(self, data, z):
    returns = data[:, :, 0]
    total_acceleration_loss = 0.0

    # Calculate acceleration loss for each lookback period
    for lookback in self.lookbacks:
      momentum_t = tf.reduce_sum(returns[:, -lookback:], axis=1)
      momentum_t_prev = tf.reduce_sum(returns[:, -(2*lookback):-lookback], axis=1) # To sum previous momentum
      acceleration = momentum_t - momentum_t_prev
      acceleration = (acceleration - tf.reduce_mean(acceleration)) / (tf.math.reduce_std(acceleration) + self.epsilon) # Normalizing
      acceleration_pred = self.acceleration_layer(z)[:, 0]
      total_acceleration_loss += tf.reduce_mean(tf.square(acceleration_pred - acceleration)) / len(self.lookbacks)

    return total_acceleration_loss

  def momentum_guidance_loss(self, z, q, momentum_scores):
    cluster_assignments = tf.argmax(q, axis=1) # Assign temporary cluster based on q
    momentum_guidance_loss = tf.constant(0.0, dtype=tf.float32)

    for cluster_idx in range(self.n_components):
      cluster_mask = tf.equal(cluster_assignments, cluster_idx)
      cluster_momentum = tf.boolean_mask(momentum_scores, cluster_mask)
      momentum_guidance_loss += tf.cond(
        tf.shape(cluster_momentum)[0] > 1,
        lambda: tf.clip_by_value(tf.math.reduce_variance(cluster_momentum) - 0.5 * tf.reduce_mean(cluster_momentum), 0.0, 50.0), # Clip for numerical stability
        lambda: tf.constant(0.0, dtype=tf.float32)
      )

    return tf.reduce_mean(momentum_guidance_loss)

  def call(self, inputs):
    encoder_outputs = self.encoder(inputs)
    z = encoder_outputs[0]
    attention_output = encoder_outputs[1]
    x_recon = self.decoder([z, attention_output])
    return x_recon, z

  def train_step(self, data):
    with tf.GradientTape() as tape:
      loss, mse_loss, cluster_loss, momentum_loss, acceleration_loss = self.compute_losses(data)
      loss = ops.mean(loss)

    grads = tape.gradient(loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    self.loss_metric.update_state(loss)
    self.mse_metric.update_state(mse_loss)
    self.cluster_metric.update_state(cluster_loss)
    self.momentum_metric.update_state(momentum_loss)
    self.acceleration_metric.update_state(acceleration_loss)
    return {m.name: m.result() for m in self.metrics}

  def test_step(self, data):
    loss, mse_loss, cluster_loss, momentum_loss, acceleration_loss = self.compute_losses(data)
    loss = ops.mean(loss)
    self.loss_metric.update_state(loss)
    self.mse_metric.update_state(mse_loss)
    self.cluster_metric.update_state(cluster_loss)
    self.momentum_metric.update_state(momentum_loss)
    self.acceleration_metric.update_state(acceleration_loss)
    return {m.name: m.result() for m in self.metrics}

  # Update cluster centroids for training purpose
  def cluster_model(self, min_samples, z_np):
    z_np_normalized = z_np / (np.std(z_np, axis=0, keepdims=True) + self.epsilon) # Normalize for stability
    hdbscan = HDBSCAN(min_cluster_size=min_samples, min_samples=min_samples, cluster_selection_method='eom')
    labels = hdbscan.fit_predict(z_np_normalized)
    probabilities = hdbscan.probabilities_

    unique_labels = np.unique(labels[labels >= 0]) # Get unique labels exclude noise
    n_components = len(unique_labels)
    if n_components == 0 or len(z_np) < min_samples:
      return np.zeros((1, self.latent_dim)), 1, 0.5

    # Get cluster centroids
    centroids = []
    for label in unique_labels:
      cluster_points = z_np[labels == label]
      cluster_probs = probabilities[labels == label]
      weights = cluster_probs / (np.sum(cluster_probs) + self.epsilon)
      centroid = np.average(cluster_points, axis=0, weights=weights)
      centroids.append(centroid)
    centroids = np.array(centroids) if centroids else np.zeros((1, self.latent_dim))

    # Random generation of centroids if fewer than max_clusters
    while len(centroids) < self.max_clusters:
      centroids = np.vstack([centroids, np.random.normal(0, 1, self.latent_dim)])
    centroids = centroids[:self.max_clusters] # Only extract centroids <= max_clusters

    weighting = 0.5 + 0.5 * min(1.0, silhouette_score(z_np_normalized[labels >= 0], labels[labels >= 0]))
    return centroids, n_components, weighting

  # Update centroids, n_components and weighting at every n epochs
  def on_epoch_end(self, epoch, logs=None):
    self.current_epoch.assign(epoch + 1)
    if self.dataset is None:
      return

    z_all = []
    for batch in self.dataset:
      mask = tf.reduce_any(batch != 0, axis=[1, 2]) if len(batch.shape) > 2 else tf.reduce_any(batch != 0, axis=1)
      encoder_outputs = self.encoder(batch)
      z = encoder_outputs[0]
      z_masked = tf.boolean_mask(z, mask)
      z_all.append(z_masked.numpy())

    z_np = np.concatenate(z_all, axis=0)

    # Skipping if no valid data for clustering
    if len(z_np) == 0 or len(z_np) < self.min_samples:
      if self.verbose == 1:
        print(f"\nEpoch {epoch + 1}: No valid data for clustering")
        return

    # Get cluster centroids
    centroids, n_components, weighting = self.cluster_model(self.min_samples, z_np)
    new_centroids = tf.convert_to_tensor(centroids, dtype=tf.float32)

    # If there are fewere clusters in the new centroids, pad the rest with random generation
    if new_centroids.shape[0] < self.max_clusters:
      padding_shape = (self.max_clusters - new_centroids.shape[0], self.latent_dim)
      padding = tf.random.normal(padding_shape, mean=0.0, stddev=1.0, dtype=tf.float32)
      new_centroids = tf.concat([new_centroids, padding], axis=0)
    elif new_centroids.shape[0] > self.max_clusters:
      new_centroids = new_centroids[:self.max_clusters]

    # Assign cluster centroids on or after pretraining ends
    if self.current_epoch == self.pretrain_epochs:
      self.n_components = n_components
      self.cluster_centroids.assign(new_centroids[:self.max_clusters])
      if self.verbose == 1:
        print(f"\nEpoch {epoch + 1}: HDBSCAN initialized {n_components} centroids")
    elif self.current_epoch > self.pretrain_epochs and (epoch + 1) % self.update_freq == 0:
      self.n_components = n_components
      updated_centroids = weighting * self.cluster_centroids + (1.0 - weighting) * new_centroids[:self.max_clusters]
      self.cluster_centroids.assign(updated_centroids)
      if self.verbose == 1:
        print(f"\nEpoch {epoch + 1}: HDBSCAN updated {n_components} centroids, silhouette weighting: {weighting:.3f}")

  @property
  def metrics(self):
    return [self.loss_metric, self.mse_metric, self.cluster_metric, self.momentum_metric, self.acceleration_metric]

# Callback class for regular update outside of training loop
class ClusterCentroidUpdateCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        self.model.on_epoch_end(epoch, logs)